In [ ]:
!pip install transformers==4.47.1 datasets accelerate

In [ ]:
!pip install protobuf sentencepiece peft wandb

In [ ]:
pip install -U "huggingface_hub[cli]"

In [ ]:
%%sh
huggingface-cli login --token hf_

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
import wandb

In [ ]:
wandb.init(project="ukrainian-to-hutsul-translation-mistralv03-synth", entity="")

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,                       
    lora_alpha=32,          
    lora_dropout=0.1,        
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, lora_config)
print("LoRA trainable parameters:")
model.print_trainable_parameters()

In [ ]:
data_files = {
    "train": "merged_final_combined.json",
    "test": "test_ivanchyk.json" 
}
dataset = load_dataset("json", data_files=data_files)


In [ ]:
def format_example(example):
    prompt = f"Translate the following Ukrainian sentence into the Hutsul dialect:\n\n{example['source']}\n\nAnswer:"
    full_text = prompt + " " + example["target"]
    return {"text": full_text}

formatted_dataset = dataset.map(format_example, remove_columns=dataset["train"].column_names)


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [ ]:
training_args = TrainingArguments(
    output_dir="./mistral-translation-lora-finetune-synt",
    overwrite_output_dir=True,
    num_train_epochs=3,                 
    per_device_train_batch_size=1,       
    gradient_accumulation_steps=64,   
    evaluation_strategy="steps",
    eval_steps=100,
    learning_rate=5e-5,                 
    weight_decay=0.01,
    fp16=True,                            
    save_total_limit=3,
    logging_steps=10,
    save_steps=100,
    report_to=["wandb"]                  
)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
class AveragingTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False,**kwargs):
        outputs = model(**inputs)
        loss = outputs.loss  
        input_ids = inputs.get("input_ids")
        if input_ids is not None:
            loss = loss / input_ids.shape[1]
        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset.get("test"),  # if evaluation data exists
    data_collator=data_collator
)

In [ ]:
trainer.train()
model.save_pretrained("./mistral-translation-lora-finetuned-s")
tokenizer.save_pretrained("./mistral-translation-lora-finetuned-s")